# Auto summarizing Text

Word importance = Word freequency

significance score = sum(word importance)

pick most significance text

### Abstract extraction

Retrive text(Download webpage or scrape) 

-------->

Process Raw text(Tokenize to sent and words, remove stopwords, )

-------->

Extract Sentences(get freequncy of words, calculate significance score, rank sent by score, pick top n sent)

In [46]:
import nltk
import urllib

from bs4 import BeautifulSoup
from collections import defaultdict
from nltk.probability import FreqDist
from heapq import nlargest
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from string import punctuation

In [4]:
article_url = 'https://timesofindia.indiatimes.com/city/kolhapur/homeopath-wife-held-in-kolhapur-for-selling-babies/articleshow/62814634.cms'

In [5]:
def getTextWaPo(url):
    page = urllib.request.urlopen(url).read().decode('utf8', 'ignore')
    soup = BeautifulSoup(page, 'lxml')
    text=''.join(map(lambda x: x.text, soup.find_all("div", {"class": "Normal"})))
    text = text.replace('\n', '').replace('  ', '')
    return text
text = getTextWaPo(article_url)

In [6]:
text

'Maharashtra police on Tuesday arrested a homeopathic doctor and his wife for allegedly selling newborns in Kolhapur district. Police said Arun Bhupal Patil sold two babies over the last three months for Rs 2 lakh each. According to Priyadarshini Chorage, district head of the child welfare committee, Patil would help unmarried women deliver children and give all the cash from the sale to the mothers.Chorage said Patil has confessed to selling a new-born delivered by a minor mother to a family in Chhattisgarh on December 23, 2017 for Rs 2 lakh. "He has also confessed to selling a new-born baby to a couple in Mumbai but has not revealed the details," she added.Police said Patil, his wife Ujwalla and a couple who took the baby had been booked under relevant sections of IPC and the Juvenile Justice Act, 2015.'

In [7]:
sentences = sent_tokenize(text)
sentences

['Maharashtra police on Tuesday arrested a homeopathic doctor and his wife for allegedly selling newborns in Kolhapur district.',
 'Police said Arun Bhupal Patil sold two babies over the last three months for Rs 2 lakh each.',
 'According to Priyadarshini Chorage, district head of the child welfare committee, Patil would help unmarried women deliver children and give all the cash from the sale to the mothers.Chorage said Patil has confessed to selling a new-born delivered by a minor mother to a family in Chhattisgarh on December 23, 2017 for Rs 2 lakh.',
 '"He has also confessed to selling a new-born baby to a couple in Mumbai but has not revealed the details," she added.Police said Patil, his wife Ujwalla and a couple who took the baby had been booked under relevant sections of IPC and the Juvenile Justice Act, 2015.']

In [8]:
words = word_tokenize(text.lower())
words

['maharashtra',
 'police',
 'on',
 'tuesday',
 'arrested',
 'a',
 'homeopathic',
 'doctor',
 'and',
 'his',
 'wife',
 'for',
 'allegedly',
 'selling',
 'newborns',
 'in',
 'kolhapur',
 'district',
 '.',
 'police',
 'said',
 'arun',
 'bhupal',
 'patil',
 'sold',
 'two',
 'babies',
 'over',
 'the',
 'last',
 'three',
 'months',
 'for',
 'rs',
 '2',
 'lakh',
 'each',
 '.',
 'according',
 'to',
 'priyadarshini',
 'chorage',
 ',',
 'district',
 'head',
 'of',
 'the',
 'child',
 'welfare',
 'committee',
 ',',
 'patil',
 'would',
 'help',
 'unmarried',
 'women',
 'deliver',
 'children',
 'and',
 'give',
 'all',
 'the',
 'cash',
 'from',
 'the',
 'sale',
 'to',
 'the',
 'mothers.chorage',
 'said',
 'patil',
 'has',
 'confessed',
 'to',
 'selling',
 'a',
 'new-born',
 'delivered',
 'by',
 'a',
 'minor',
 'mother',
 'to',
 'a',
 'family',
 'in',
 'chhattisgarh',
 'on',
 'december',
 '23',
 ',',
 '2017',
 'for',
 'rs',
 '2',
 'lakh',
 '.',
 '``',
 'he',
 'has',
 'also',
 'confessed',
 'to',
 'sel

In [9]:
_stopwords = set(stopwords.words('english') + list(punctuation))
_stopwords

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'need

In [10]:
word_sent = [word for word in words if word not in _stopwords]
word_sent

['maharashtra',
 'police',
 'tuesday',
 'arrested',
 'homeopathic',
 'doctor',
 'wife',
 'allegedly',
 'selling',
 'newborns',
 'kolhapur',
 'district',
 'police',
 'said',
 'arun',
 'bhupal',
 'patil',
 'sold',
 'two',
 'babies',
 'last',
 'three',
 'months',
 'rs',
 '2',
 'lakh',
 'according',
 'priyadarshini',
 'chorage',
 'district',
 'head',
 'child',
 'welfare',
 'committee',
 'patil',
 'would',
 'help',
 'unmarried',
 'women',
 'deliver',
 'children',
 'give',
 'cash',
 'sale',
 'mothers.chorage',
 'said',
 'patil',
 'confessed',
 'selling',
 'new-born',
 'delivered',
 'minor',
 'mother',
 'family',
 'chhattisgarh',
 'december',
 '23',
 '2017',
 'rs',
 '2',
 'lakh',
 '``',
 'also',
 'confessed',
 'selling',
 'new-born',
 'baby',
 'couple',
 'mumbai',
 'revealed',
 'details',
 "''",
 'added.police',
 'said',
 'patil',
 'wife',
 'ujwalla',
 'couple',
 'took',
 'baby',
 'booked',
 'relevant',
 'sections',
 'ipc',
 'juvenile',
 'justice',
 'act',
 '2015']

In [11]:
from nltk.probability import FreqDist
freq = FreqDist(word_sent)
freq

FreqDist({"''": 1,
          '2': 2,
          '2015': 1,
          '2017': 1,
          '23': 1,
          '``': 1,
          'according': 1,
          'act': 1,
          'added.police': 1,
          'allegedly': 1,
          'also': 1,
          'arrested': 1,
          'arun': 1,
          'babies': 1,
          'baby': 2,
          'bhupal': 1,
          'booked': 1,
          'cash': 1,
          'chhattisgarh': 1,
          'child': 1,
          'children': 1,
          'chorage': 1,
          'committee': 1,
          'confessed': 2,
          'couple': 2,
          'december': 1,
          'deliver': 1,
          'delivered': 1,
          'details': 1,
          'district': 2,
          'doctor': 1,
          'family': 1,
          'give': 1,
          'head': 1,
          'help': 1,
          'homeopathic': 1,
          'ipc': 1,
          'justice': 1,
          'juvenile': 1,
          'kolhapur': 1,
          'lakh': 2,
          'last': 1,
          'maharashtra': 1,
    

In [12]:
from heapq import nlargest

In [13]:
nlargest(10, freq, key=freq.get)

['patil',
 'selling',
 'said',
 'police',
 'wife',
 'district',
 'rs',
 '2',
 'lakh',
 'confessed']

In [14]:
from collections import defaultdict
ranking = defaultdict(int)

for i, sent in enumerate(sentences):
    for w in word_tokenize(sent.lower()):
        if w in freq:
            ranking[i] += freq[w]
            
ranking

defaultdict(int, {0: 17, 1: 23, 2: 51, 3: 41})

In [15]:
sent_idx = nlargest(2, ranking, key=ranking.get)
sent_idx

[2, 3]

In [16]:
summry_list = [sentences[j] for j in sorted(sent_idx)]

In [17]:
def summarize(text, n):
    sentences = sent_tokenize(text)
    
    assert n < len(sentences)
    words = word_tokenize(text.lower())
    _stopwords = set(stopwords.words('english') + list(punctuation))
    
    word_sent = [word for word in words if word not in _stopwords]
    freq = FreqDist(word_sent)
    
    ranking = defaultdict(int)

    for i, sent in enumerate(sentences):
        for w in word_tokenize(sent.lower()):
            if w in freq:
                ranking[i] += freq[w]
                
    sent_idx = nlargest(2, ranking, key=ranking.get)
    return [sentences[j] for j in sorted(sent_idx)]



In [18]:
''.join(summarize(text, 2))

'According to Priyadarshini Chorage, district head of the child welfare committee, Patil would help unmarried women deliver children and give all the cash from the sale to the mothers.Chorage said Patil has confessed to selling a new-born delivered by a minor mother to a family in Chhattisgarh on December 23, 2017 for Rs 2 lakh."He has also confessed to selling a new-born baby to a couple in Mumbai but has not revealed the details," she added.Police said Patil, his wife Ujwalla and a couple who took the baby had been booked under relevant sections of IPC and the Juvenile Justice Act, 2015.'

## ML Approach

feature extraction using bag of words model

K-means clustering to identify set of topics

k-nearest neighbours model for classifying text into topics

Corpus of article

----------->

identify themes

----------->

Assign themes to article

In [35]:
def getAllDoxyDonkeyPosts(url, links):
    # request = urllib.request(url)
    response = urllib.request.urlopen(url)
    soup = BeautifulSoup(response)
    for a in soup.findAll('a'):
        try:
            url = a['href']
            title = a['title']
            if title == 'Older Posts':
                print (title, url)
                links.append(url)
                getAllDoxyDonkeyPosts(url, links)
        except:
            title = ""
    return

blogurl = 'http://doxydonkey.blogspot.in'
links = []
getAllDoxyDonkeyPosts(blogurl, links)

c:\users\bray1\projects\nlp\env\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file c:\python36\Lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-05-23T19:53:00-07:00&max-results=7
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-05-14T19:02:00-07:00&max-results=7&start=7&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-05-02T19:43:00-07:00&max-results=7&start=14&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-04-17T19:26:00-07:00&max-results=7&start=21&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-04-10T18:56:00-07:00&max-results=7&start=28&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-03-30T19:57:00-07:00&max-results=7&start=35&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-03-20T19:47:00-07:00&max-results=7&start=42&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-03-02T17:42:00-08:00&max-results=7&start=49&by-date=false
Older Posts http://doxydonkey.bl

Older Posts http://doxydonkey.blogspot.in/search?updated-max=2015-04-23T20:19:00-07:00&max-results=7&start=462&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2015-04-14T19:40:00-07:00&max-results=7&start=469&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2015-04-05T20:22:00-07:00&max-results=7&start=476&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2015-03-24T20:12:00-07:00&max-results=7&start=483&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2015-03-15T20:41:00-07:00&max-results=7&start=490&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2015-03-03T19:30:00-08:00&max-results=7&start=497&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2015-02-22T19:55:00-08:00&max-results=7&start=504&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2015-02-11T20:02:00-08:00&max-results=7&start=511&by-date=false


In [37]:
def getDoxyDonkeyText(testurl):
    response = urllib.request.urlopen(testurl)
    soup = BeautifulSoup(response)
    mydivs = soup.findAll('div', {'class': 'post-body'})
    posts = []
    for div in mydivs:
        posts += map(lambda p: p.text.replace('  ', ''), div.findAll('li'))
    return posts

data = getDoxyDonkeyText(blogurl)

c:\users\bray1\projects\nlp\env\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file c:\python36\Lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [38]:
data

['Alibaba Falters in Bid to Take Mobile Phone Control From Tencent:\xa0Alibaba Group Holding Ltd.’s six-year-old excursion into mobile operating systems is faltering in China, casting doubt over software that bears\xa0billionaire-founder\xa0Jack Ma’s name and was once touted as key to countering Tencent Holdings Ltd.\xa0China’s largest e-commerce company debuted YunOS in 2011, a system that underpins search, shopping and browsing that its executives last year said could attain as much as 25 percent domestic market share by the end of 2016 -- surpassing Apple Inc.’s iOS. Six years on, YunOS’ slice of China software installations stands at just 2.2 percent while its share of 2016 shipments was 10 percent, researchers Canalys and Counterpoint estimate, respectively. Alibaba disputes those numbers.\xa0Alibaba managers have grown increasingly unhappy with its sluggish adoption and have begun an internal debate around the software’s future, a person familiar with the matter said. No conclusi

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [40]:
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words="english")
x = vectorizer.fit_transform(data)
x[0]

<1x229 sparse matrix of type '<class 'numpy.float64'>'
	with 33 stored elements in Compressed Sparse Row format>

In [41]:
print(x[0])

  (0, 127)	0.3495108507283361
  (0, 47)	0.11650361690944537
  (0, 195)	0.3495108507283361
  (0, 86)	0.08697311064905155
  (0, 38)	0.2842219494516761
  (0, 181)	0.2842219494516761
  (0, 109)	0.10424734943869328
  (0, 95)	0.11650361690944537
  (0, 112)	0.09474064981722537
  (0, 41)	0.10424734943869328
  (0, 167)	0.11650361690944537
  (0, 176)	0.23300723381889074
  (0, 162)	0.2241505295348984
  (0, 6)	0.11650361690944537
  (0, 139)	0.2609193319471547
  (0, 122)	0.09474064981722537
  (0, 173)	0.23300723381889074
  (0, 5)	0.23300723381889074
  (0, 227)	0.08697311064905155
  (0, 107)	0.07471684317829946
  (0, 1)	0.11650361690944537
  (0, 20)	0.11650361690944537
  (0, 81)	0.10424734943869328
  (0, 71)	0.10424734943869328
  (0, 123)	0.20849469887738656
  (0, 190)	0.11650361690944537
  (0, 103)	0.11650361690944537
  (0, 60)	0.11650361690944537
  (0, 16)	0.11650361690944537
  (0, 137)	0.11650361690944537
  (0, 124)	0.11650361690944537
  (0, 55)	0.11650361690944537
  (0, 171)	0.10424734943869328


In [43]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3, init="k-means++", max_iter=100, n_init=1, verbose=True)
km

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=3, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=True)

In [44]:
km.fit(x)

Initialization complete
Iteration  0, inertia 21.154
Iteration  1, inertia 11.234
Converged at iteration 1: center shift 0.000000e+00 within tolerance 3.686399e-07


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=3, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=True)

In [45]:
import numpy as np
np.unique(km.labels_, return_counts=True)

(array([0, 1, 2]), array([6, 6, 4]))

In [57]:
text = {}
for i, cluster in enumerate(km.labels_):
    oneDocument = data[i]
    if cluster not in text.keys():
        text[cluster] = oneDocument
    else:
        text[cluster] += oneDocument

In [58]:
_stopwords = set(stopwords.words('english') + list(punctuation) + ['million', 'billion', 'year', 'millions', 'billions', 'y/y', "'s", "'"])

In [59]:
keywords = {}
counts = {}
for cluster in range(3):
    word_sent = word_tokenize(text[cluster].lower())
    word_sent = [word for word in word_sent if word not in _stopwords]
    freq = FreqDist(word_sent)
    keywords[cluster] = nlargest(100, freq, key=freq.get)
    counts[cluster] = freq

In [60]:
unique_keys = {}
for cluster in range(3):
    other_clusters = list(set(range(3)) - set([cluster]))
    
    keys_other_cluster = set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))
    unique = set(keywords[cluster]) - keys_other_cluster
    unique_keys[cluster] = nlargest(10, unique, key=counts[cluster].get)


In [61]:
unique_keys

{0: ['drivers',
  'amazon',
  'drone',
  'bitcoin',
  'inc.',
  'global',
  'capital',
  'spark',
  'hit',
  'time'],
 1: ['china',
  'car',
  'india',
  'electric',
  'google',
  'government',
  'vehicles',
  'chinese',
  'states',
  'software'],
 2: ['pinterest',
  'round',
  'startup',
  '1',
  'casper',
  'funding',
  'target',
  'snapdeal',
  'investment',
  'according']}

In [62]:
 article = "Facebook Inc provided more evidence on Wednesday that it can turn eyeballs into profit as the maker of the world's most popular app and social website trounced Wall Street's estimates, sending its shares to an all-time high. The leading social media company's mobile app and push into video attracted new advertisers and encouraged existing ones to spend more. It now has more than 1.7 billion monthly users, well ahead of any rivals. Its shares were up 5.4 percent in after-hours trading at $130.01, after hitting their highest since the company went public in 2012. Mobile advertising revenue accounted for 84 percent of the company's total advertising revenue, compared with 76 percent a year earlier. Total advertising revenue surged 63 percent to $6.24 billion, beating the average analyst estimate of $5.80 billion.Meanwhile, Facebook still has several untapped areas for revenue opportunities, including its WhatsApp and Messenger apps, both of which have more than 1 billion users. But Wehner said the company does not plan to monetize them any time soon, and that it is instead focused on building interactions between businesses and users on the apps. Facebook also owns picture-sharing app Instagram, which recently announced it has more than 500 million users. Facebook has yet to say how much money Instagram makes, but research firm eMarketer predicts it will make $1.5 billion in revenue this year.Total revenue rose 59.2 percent to $6.44 billion, ahead of analysts' average estimate of $6.02 billion."

In [75]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=10)
classifier.fit(x, km.labels_)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [76]:
test = vectorizer.transform([article])
test

<1x229 sparse matrix of type '<class 'numpy.float64'>'
	with 30 stored elements in Compressed Sparse Row format>

In [77]:
classifier.predict(test)

array([0])